In [12]:
# data wrangling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
# Read data
data_dir = 'C:/Users/Dell/kaggle_dataset'
df = pd.read_csv(data_dir+'/spam.csv', encoding='latin-1')

# Train test split
data_train, data_test,labels_train, labels_test = train_test_split(df.v2, df.v1, test_size=0.2, random_state=0)
print(data_train[:10])
print(labels_train[:10])

1114    No no:)this is kallis home ground.amla home to...
3589    I am in escape theatre now. . Going to watch K...
3095    We walked from my moms. Right on stagwood pass...
1012       I dunno they close oredi not... ÌÏ v ma fan...
3320                               Yo im right by yo work
4130    \Its Ur luck to Love someone. Its Ur fortune t...
1197     He also knows about lunch menu only da. . I know
5426        Oh yeah! And my diet just flew out the window
624     Nah it's straight, if you can just bring bud o...
2260    SplashMobile: Choose from 1000s of gr8 tones e...
Name: v2, dtype: object
1114     ham
3589     ham
3095     ham
1012     ham
3320     ham
4130     ham
1197     ham
5426     ham
624      ham
2260    spam
Name: v1, dtype: object


In [23]:
# Find the unique vocabulary
def GetVocabulary(data):
    vocab_dict = {}
    wid = 0
    for document in data:
        words = document.split()
        for word in words:
            word = word.lower()
            if word not in vocab_dict:
                vocab_dict[word] = wid
                wid+=1
    return vocab_dict

# 用训练集建立词汇表
vocab_dict = GetVocabulary(data_train)
print('Number of unique words:' + str(len(vocab_dict.keys())))

Number of unique words:11706


In [64]:
# 词汇表向量统计
def Document2Vector(vocab_dict, data):
    word_vector = np.zeros(len(vocab_dict.keys()))
    words = data.split()
    out_of_voc = 0
    for word in words:
        word = word.lower()
        if word in vocab_dict:
            word_vector[vocab_dict[word]] += 1
        else:
            out_of_voc += 1
    return word_vector, out_of_voc

train_matrix = []
for document in data_train.values:
    word_vector,_ = Document2Vector(vocab_dict, document)
    train_matrix.append(word_vector)
print(len(train_matrix))

4457


In [67]:
def NaiveBayes_train(train_matrix, labels_train):
    num_docs = len(train_matrix)
    num_words = len(train_matrix[0])
    
    spam_word_counter = np.ones(num_words)
    ham_word_counter = np.ones(num_words)
    
    spam_total_count = 0
    ham_total_count = 0
    
    spam_count = 0
    ham_count = 0
    
    for i in range(num_docs):
        if i%500 == 0:
            print('Train on the doc id'+str(i))
        if labels_train[i] == 'ham':
            spam_word_counter += train_matrix[i]
            spam_total_count += sum(train_matrix[i])
            spam_count += 1
        else:
            ham_word_counter += train_matrix[i]
            ham_total_count += sum(train_matrix[i])
            ham_count += 1
    
    p_spam_vector = np.log(spam_word_counter/(spam_total_count + num_words))
    p_ham_vector = np.log(ham_word_counter/(ham_total_count + num_words))
    
    return p_spam_vector, np.log(spam_count/num_docs), p_ham_vector, np.log(ham_count/num_docs), spam_total_count, ham_total_count
# Train data as a model
p_spam_vector, np.log(spam_count/num_docs), p_ham_vector, np.log(ham_count/num_docs), spam_total_count, ham_total_count = NaiveBayes_train(train_matrix, labels_train.values)


SyntaxError: cannot assign to function call (<ipython-input-67-ff2c93b37aad>, line 31)

In [68]:
def Predict(test_word_vector, p_spam_vector, p_spam, p_ham_vector, p_ham, spam_smoothing, ham_smoothing):
    spam = sum(test_word_vector * p_spam_vector) + p_spam + spam_smoothing
    ham = sum(test_word_vector * p_ham_vector) + p_ham + ham_smoothing
    if spam > ham:
        return 'spam'
    else:
        return 'ham'

num_words = len(vocab_dict.keys())
prediction = []
i = 0

for document in data_test.values:
    if i%200 == 0:
        print('Test on the doc id:' + str(i))
    i += 1
    test_word_vector, out_of_vector = Document2Vector(vocab_dict, document)
    
    if out_of_vector != 0:
        spam_smoothing = np.log(out_of_vector/(spam_total_count+num_words))
        ham_smoothing = np.log(out_of_vector/(ham_total_count+num_words))
    
    else:
        spam_smoothing = 0
        ham_smoothing = 0
    anw = Predict(test_word_vector, p_spam_vector, p_spam, p_ham_vector, p_ham, spam_smoothing, ham_smoothing)
    prediction.append(anw)

len(prediction)

Test on the doc id:0
Test on the doc id:200
Test on the doc id:400
Test on the doc id:600
Test on the doc id:800
Test on the doc id:1000


1115

In [72]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
print(accuracy_score(labels_test, prediction))
print(classification_report(labels_test, prediction))
print(confusion_matrix(labels_test, prediction))

0.97847533632287
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       949
        spam       0.98      0.87      0.92       166

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115

[[946   3]
 [ 21 145]]
